1. going to fix image preprocessing
2. going to use pretrained featurse kfold
3. going to re-ensemble

this should produce a better leaderboard score

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6846146967892223123, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 11332668621
 locality {
   bus_id: 1
 }
 incarnation: 13120922055316095303
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:04:00.0"]

In [2]:
import os, glob, bcolz, gc

import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy import ndimage, misc

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score

from keras import backend as K
from keras import optimizers
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers.convolutional import MaxPooling2D, Convolution2D
from keras.layers.advanced_activations import PReLU, LeakyReLU

from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.inception_v3 import preprocess_input as preprocess_input_incep_xcep
from keras.applications.imagenet_utils import preprocess_input as preprocess_input_vgg_resnet

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
def read_img(img_path, img_shape):
    img = misc.imread(img_path)
    img = misc.imresize(img, img_shape)
    return img

def read_imgs(img_height, img_width):
    train_img, test_img = [],[]
    for img_path in tqdm(train_set['name'].iloc[:]):
        train_img.append(read_img(os.path.join(path, 'train', str(img_path)+'.jpg'), (img_height, img_width)))

    for img_path in tqdm(test_set['name'].iloc[:]):
        test_img.append(read_img(os.path.join(path, 'test', str(img_path)+'.jpg'), (img_height, img_width)))
    return np.array(train_img), np.array(test_img)

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

def freeze_model(model):
    for layer in model.layers:
        layer.trainable = False
    return model

def grab_optimizer(opt, lr):
    if opt == 'sgd':
        return optimizers.SGD(lr=lr, decay=1e-6, momentum=0.8, nesterov=True)
    elif opt == 'adam':
        return optimizers.Adam(lr=lr)
    elif opt == 'adagrad':
        return optimizers.Adagrad(lr=lr)
    elif opt == 'rmsprop':
        return optimizers.RMSprop(lr=lr)
    
def dense_block(units, activation, drop_prob, inputs):
    x = Dense(units, activation=None)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Dropout(drop_prob)(x)
    return x

def make_conv_model(input_shape, optimizer):
    inputs = Input(shape=input_shape)
    m = conv_block(16, (3,3), (2,2),'relu', inputs=inputs)
    m = conv_block(32, (3,3), (2,2), 'relu', inputs=m)
    m = conv_block(64, (3,3), (2,2), 'relu', inputs=m)
    m = conv_block(128, (3,3), (2,2), 'relu', inputs=m)
    m = conv_block(256, (3,3), (2,2), 'relu', inputs=m)
    m = Flatten()(m)
    m = dense_block(2048, 'relu', 0.25, inputs=m)
    m = dense_block(512, 'relu', 0.5, inputs=m)
    outputs = dense_block(1, 'sigmoid', 0, inputs=m)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def make_vgg19_conv(input_shape):
    base_model = VGG19(input_shape=input_shape, weights='imagenet', include_top=False)
    base_model = freeze_model(base_model)
    return base_model

def make_incepv3_conv(input_shape):
    base_model = InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False)
    base_model = freeze_model(base_model)
    return base_model

def make_resnet50_conv(input_shape):
    base_model = ResNet50(input_shape=input_shape, weights='imagenet', include_top=False)
    base_model = freeze_model(base_model)
    return base_model

def make_xception_conv(input_shape):
    base_model = Xception(input_shape=input_shape, weights='imagenet', include_top=False)
    base_model = freeze_model(base_model)
    return base_model

def make_ft_dense(input_shape, optimizer):
    inputs = Input(shape=input_shape)
    m = Flatten()(inputs)
    m = dense_block(1024, 'relu', 0.25, inputs=m)
    m = dense_block(1024, 'relu', 0.5, inputs=m)
    outputs = dense_block(1, 'sigmoid', 0, inputs=m)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def train_model_k_folds_ft(model, base_model, train_data, train_label, model_out, model_init_weights, epochs, kfolds, batch_size):
    kf = KFold(n_splits=kfolds, shuffle=True)
    
    i = 0
    models_stats = {}
    for train_ixs, valid_ixs in kf.split(train_data):
        # use convolutional model to precompute training
        # features with data aug
    
        x_train = train_data[train_ixs]
        x_valid = train_data[valid_ixs]
        y_train = train_label[train_ixs]
        y_valid = train_label[valid_ixs]

        x_train = base_model.predict(x_train)
        x_valid = base_model.predict(x_valid)
                
        #re-initialzie the weights of the model on each run
        model.load_weights(model_init_weights)
        model_out_file = '{}_{}.model'.format(model_out, str(i))
        model_checkpoint = ModelCheckpoint(model_out_file, 
                                            monitor='val_loss', 
                                            save_best_only=True)
        
        print('training dense layers:')
        model.fit(x=x_train, y=y_train, 
                  batch_size=batch_size,
                  validation_data=(x_valid,y_valid),
                  epochs=epochs,
                  verbose=1,
                  callbacks=[model_checkpoint])
        
        model.load_weights(model_out_file)
        
        eval_tr = model.evaluate(x_train, y_train)
        eval_va = model.evaluate(x_valid, y_valid)
        
        tr_score = roc_auc_score(y_train, model.predict(x_train)[:, 0])
        va_score = roc_auc_score(y_valid, model.predict(x_valid)[:, 0])
        
        print('\n')
        print('kfold: {}'.format(str(i)))
        print('best model train acc: {}, loss: {}'.format(eval_tr[1], eval_tr[0]))
        print('best model valid acc: {}, loss: {}'.format(eval_va[1], eval_va[0]))
        print('best model train aroc score: {}, valid aroc score: {}'.format(tr_score, va_score))
        print('\n')
        models_stats[model_out_file] = {'score_tr_va':[tr_score, va_score], 'train_acc_loss':[eval_tr[1], eval_tr[0]], 'val_acc_loss':[eval_va[1], eval_va[0]]}
        
        with open(os.path.join(models_path,'{}_{}.out'.format(model_out,'history')), 'a') as f:
            f.write('kfold: {}'.format(str(i)))
            f.write('best model train acc: {}, loss: {}'.format(eval_tr[1], eval_tr[0]))
            f.write('best model valid acc: {}, loss: {}'.format(eval_va[1], eval_va[0]))
            f.write('best model train aroc score: {}, valid aroc score: {}'.format(tr_score, va_score))
            f.write('\n')
        
        i += 1
    
    return models_stats

setup data dirs and read in imgs:

In [4]:
DATA_DIR = os.path.join('/scratch', 'yns207', 'data_invasive')

path = DATA_DIR
test_path = os.path.join(path, 'test')
models_path = os.path.join(path, 'results')
train_path = os.path.join(path, 'train')
valid_path = os.path.join(path, 'valid')
print('DATA_PATH: ' + path)

DATA_PATH: /scratch/yns207/data_invasive


# loading data

In [ ]:
train_set = pd.read_csv(os.path.join(path, 'train_labels.csv'))
test_set = pd.read_csv(os.path.join(path, 'sample_submission.csv'))

In [ ]:
train_img, test_img = read_imgs(300,400)
train_label = np.array(train_set['invasive'].iloc[:])

create a holdout set of 10%:

In [ ]:
train_img, hold_img, train_labels, hold_labels = train_test_split(train_img, train_label, test_size=0.10)

In [ ]:
train_img.shape, hold_img.shape, train_labels.shape, hold_labels.shape

save the datasets unaltered so they can be loaded again at a later point:

In [ ]:
%cd $path
save_array('aug_3_train_img.dat', train_img)
save_array('aug_3_hold_img.dat', hold_img)
save_array('aug_3_train_labels.dat', train_labels)
save_array('aug_3_hold_labels.dat', hold_labels)

# checkpoint

read the datasets with bcolz:

In [5]:
%cd $path
train_img = load_array('aug_3_train_img.dat')
hold_img = load_array('aug_3_hold_img.dat')
train_labels = load_array('aug_3_train_labels.dat')
hold_labels = load_array('aug_3_hold_labels.dat')

/scratch/yns207/data_invasive


In [6]:
train_img.shape, hold_img.shape, train_labels.shape, hold_labels.shape

((2065, 300, 400, 3), (230, 300, 400, 3), (2065,), (230,))

In [8]:
batch_size = 32
epochs = 100
kfolds = 5
lr = 0.00025

In [ ]:
# get it into right folder
%cd $models_path
model_name = 'invasive_vgg19_aug3'
init_weights_model = '{}_base.model'.format(model_name)

# create base model
base_model = make_vgg19_conv(train_img[0].shape)

# train dense model ontop of precomputed conv features
dense_model = make_ft_dense(input_shape=tuple(base_model.output[0].shape.as_list()), optimizer=grab_optimizer('adam', lr))
dense_model.save_weights(init_weights_model)

#preprocess imgs
vgg19_train_img = preprocess_input_vgg_resnet(train_img.astype(np.float32))

# train dense model on folds
performance1 = train_model_k_folds_ft(dense_model, base_model, vgg19_train_img, train_labels, model_name, init_weights_model, epochs, kfolds, batch_size)

In [ ]:
#resnet50
%cd $models_path
model_name = 'invasive_resnet50_aug3'
init_weights_model = '{}_base.model'.format(model_name)

base_model = make_resnet50_conv(train_img[0].shape)
dense_model = make_ft_dense(input_shape=tuple(base_model.output[0].shape.as_list()), optimizer=grab_optimizer('adam', lr))
dense_model.save_weights(init_weights_model)

# preprocess (demean normalize, etc for vgg)
resnet50_train_img = preprocess_input_vgg_resnet(train_img.astype(np.float32))
performance2 = train_model_k_folds_ft(dense_model, base_model, resnet50_train_img, train_labels, model_name, init_weights_model, epochs, kfolds, batch_size)

In [ ]:
#inceptionv3
%cd $models_path
model_name = 'invasive_inceptionv3_aug3'
init_weights_model = '{}_base.model'.format(model_name)

base_model = make_incepv3_conv(train_img[0].shape)
dense_model = make_ft_dense(input_shape=tuple(base_model.output[0].shape.as_list()), optimizer=grab_optimizer('adam', lr))
dense_model.save_weights(init_weights_model)

inceptionv3_train_img = preprocess_input_incep_xcep(train_img.astype(np.float32))
performance3 = train_model_k_folds_ft(dense_model, base_model, inceptionv3_train_img, train_labels, model_name, init_weights_model, epochs, kfolds, batch_size)

In [ ]:
%cd $models_path
model_name = 'invasive_xception_aug3'
init_weights_model = '{}_base.model'.format(model_name)

base_model = make_xception_conv(train_img[0].shape)
dense_model = make_ft_dense(input_shape=tuple(base_model.output[0].shape.as_list()), optimizer=grab_optimizer('adam', lr))
dense_model.save_weights(init_weights_model)

xception_train_img = preprocess_input_incep_xcep(train_img.astype(np.float32))
performance4 = train_model_k_folds_ft(dense_model, base_model, xception_train_img, train_labels, model_name, init_weights_model, epochs, kfolds, batch_size)

In [ ]:
performance1

In [ ]:
performance2

In [ ]:
performance3

In [ ]:
performance4

# summary
the 10% holdout didnt work so we want to train the model on the holdout data
we will use the training data (even though some of it was trained on) as a pseudo validation set
if this was from th estart it would eb a big nono but we dont really care here, we just want some
idea that we arent overfitting the 10% holdout

In [8]:
# inceptionv3_aug3 ,fold 3
model_name_new = 'invasive_inceptionv310p_aug6'
model_name = 'invasive_inceptionv3_aug3'

# preprocess data:
inceptionv3_hold_img = preprocess_input_incep_xcep(hold_img.astype(np.float32))
inceptionv3_train_img = preprocess_input_incep_xcep(train_img.astype(np.float32))
    
# make model:
base_model = make_incepv3_conv(hold_img[0].shape)

#precompute features
x_hold = base_model.predict(inceptionv3_hold_img)
x_train = base_model.predict(inceptionv3_train_img)

# for the right folds load the right models
# 0 ,1 ,2 ,3 ,4
for i in [3]:
    model_out_file = '{}_{}.model'.format(model_name_new, i)
    model_weights = '{}_{}.model'.format(model_name, i)
    
    #make dense model
    dense_model = make_ft_dense(input_shape=tuple(base_model.output[0].shape.as_list()), optimizer=grab_optimizer('adam', lr))
    dense_model.load_weights(model_weights)
    
    # train dense model
    model_checkpoint = ModelCheckpoint(model_out_file, monitor='val_loss', save_best_only=True)
    
    hist = dense_model.fit(x=x_hold, y=hold_labels, 
                  batch_size=batch_size,
                  validation_data=(x_train,train_labels),
                  epochs=15,
                  verbose=1,
                  callbacks=[model_checkpoint])
    
    dense_model.load_weights(model_out_file)
    tr_score = roc_auc_score(hold_labels, dense_model.predict(x_hold)[:, 0])
    va_score = roc_auc_score(train_labels, dense_model.predict(x_train)[:, 0])
    
    print('train score, valid score:')
    print(tr_score, va_score)

Train on 230 samples, validate on 2065 samples
Epoch 1/15
230/230 [==============================] - 9s - loss: 0.2067 - acc: 0.9304 - val_loss: 0.1133 - val_acc: 0.9908
Epoch 2/15
230/230 [==============================] - 4s - loss: 0.1310 - acc: 0.9826 - val_loss: 0.1167 - val_acc: 0.9913

In [12]:
K.clear_session()
gc.collect()

49

In [9]:
#xception_aug3 folds 1 ,2 ,3
batch_size = 16
model_name_new = 'invasive_xception10p_aug6'
model_name = 'invasive_xception_aug3'

# preprocess data:
inceptionv3_hold_img = preprocess_input_incep_xcep(hold_img.astype(np.float32))
inceptionv3_train_img = preprocess_input_incep_xcep(train_img.astype(np.float32))
    
# make model:
base_model = make_xception_conv(hold_img[0].shape)

#precompute features
x_hold = base_model.predict(inceptionv3_hold_img)
x_train = base_model.predict(inceptionv3_train_img)

# for the right folds load the right models
# 0 ,1 ,2 ,3 ,4
for i in [1,2,3]:
    model_out_file = '{}_{}.model'.format(model_name_new, i)
    model_weights = '{}_{}.model'.format(model_name, i)
    
    #make dense model
    dense_model = make_ft_dense(input_shape=tuple(base_model.output[0].shape.as_list()), optimizer=grab_optimizer('adam', lr))
    dense_model.load_weights(model_weights)
    
    # train dense model
    model_checkpoint = ModelCheckpoint(model_out_file, monitor='val_loss', save_best_only=True)
    
    hist = dense_model.fit(x=x_hold, y=hold_labels, 
                  batch_size=batch_size,
                  validation_data=(x_train,train_labels),
                  epochs=15,
                  verbose=1,
                  callbacks=[model_checkpoint])
    
    dense_model.load_weights(model_out_file)
    tr_score = roc_auc_score(hold_labels, dense_model.predict(x_hold)[:, 0])
    va_score = roc_auc_score(train_labels, dense_model.predict(x_train)[:, 0])
    
    print('train score, valid score:')
    print(tr_score, va_score)
    K.clear_session()
    gc.collect()

Train on 230 samples, validate on 2065 samples
Epoch 1/15
230/230 [==============================] - 18s - loss: 0.2011 - acc: 0.9304 - val_loss: 0.1075 - val_acc: 0.9908